## 问题定义

本问题为求解2D介质板波导中的电磁波传播。

其几何如下所示：

![problem](./resource/problem.png)

整个区域为$[0, 2] \times [0, 2]$的矩形。空间真空，相对介电常数为1.左侧为波导端口，右侧为吸收边界（absorbing boundary, ABC），顶部和底部为理想导电体（Perfect electronic conductor, PEC）。在中间位置有一个水平的介质板。介质板的相对介电常数为2。

对于上述定义有如下的PDE和定解条件：

![problem](./resource/equation.png)


### 求解目标

要求求解上述求解区域中的$E_z(x,y)$


## 求解

In [1]:
import os
import warnings

# optional
# set appropriate GPU in case of multi-GPU machine
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [2]:
from modulus.sym.hydra import to_yaml
from modulus.sym.hydra.utils import compose

In [3]:
from sympy import Symbol, Eq, Heaviside, sqrt
from sympy.logic.boolalg import Or
import numpy as np
from scipy.sparse.linalg import eigsh
from scipy.sparse import diags

import modulus.sym
from modulus.sym.hydra import instantiate_arch, ModulusConfig
from modulus.sym.solver import Solver
from modulus.sym.domain import Domain
from modulus.sym.geometry.primitives_2d import Rectangle
from modulus.sym.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
    PointwiseConstraint,
)
from modulus.sym.domain.validator import PointwiseValidator
from modulus.sym.domain.inferencer import VoxelInferencer
from modulus.sym.utils.io.plotter import ValidatorPlotter, InferencerPlotter
from modulus.sym.key import Key
from modulus.sym.eq.pdes.wave_equation import HelmholtzEquation
from modulus.sym.eq.pdes.navier_stokes import GradNormal

In [4]:
cfg = compose(config_path="conf", config_name="config")
cfg.network_dir = 'outputs'    # Set the network directory for checkpoints
print(to_yaml(cfg))

/usr/local/lib/python3.10/dist-packages/modulus/sym/hydra/utils.py:148: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(


training:
  max_steps: 500000
  grad_agg_freq: 1
  rec_results_freq: 1000
  rec_validation_freq: ${training.rec_results_freq}
  rec_inference_freq: ${training.rec_results_freq}
  rec_monitor_freq: ${training.rec_results_freq}
  rec_constraint_freq: ${training.rec_results_freq}
  save_network_freq: 1000
  print_stats_freq: 100
  summary_freq: 1000
  amp: false
  amp_dtype: float16
  ntk:
    use_ntk: false
    save_name: null
    run_freq: 1000
graph:
  func_arch: false
  func_arch_allow_partial_hessian: true
stop_criterion:
  metric: null
  min_delta: null
  patience: 50000
  mode: min
  freq: 1000
  strict: false
profiler:
  profile: false
  start_step: 0
  end_step: 100
  name: nvtx
network_dir: outputs
initialization_network_dir: ''
save_filetypes: vtk,npz
summary_histograms: false
jit: true
jit_use_nvfuser: true
jit_arch_mode: only_activation
jit_autograd_nodes: false
cuda_graphs: true
cuda_graph_warmup: 20
find_unused_parameters: false
broadcast_buffers: false
device: ''
debug: fa

### 定义必要组件

In [5]:
def Laplacian_1D_eig(a, b, N, eps=lambda x: np.ones_like(x), k=3):  # 计算拉普拉斯特征值
    n = N - 2
    h = (b - a) / (N - 1)

    L = diags([1, -2, 1], [-1, 0, 1], shape=(n, n))
    L = -L / h**2

    x = np.linspace(a, b, num=N)
    M = diags([eps(x[1:-1])], [0])

    eigvals, eigvecs = eigsh(L, k=k, M=M, which="SM")
    eigvecs = np.vstack((np.zeros((1, k)), eigvecs, np.zeros((1, k))))
    norm_eigvecs = np.linalg.norm(eigvecs, axis=0)
    eigvecs /= norm_eigvecs
    return eigvals.astype(np.float32), eigvecs.astype(np.float32), x.astype(np.float32)

In [6]:
x, y = Symbol("x"), Symbol("y")

height = 2  # 区域长度
width = 2  # 区域宽度

len_slab = 0.6  # 介质板宽度
eps0 = 1.0  # 空间的相对介电常数
eps1 = 2.0  # 介质板的相对介电常数
eps_numpy = lambda y: np.where(
    np.logical_and(y > (height - len_slab) / 2, y < (height + len_slab) / 2),
    eps1,
    eps0,
)  # 定义整个区域的相对介电常数（np.array）
eps_sympy = sqrt(
    eps0
    + (
        Heaviside(y - (height - len_slab) / 2)
        - Heaviside(y - (height + len_slab) / 2)
    )
    * (eps1 - eps0)
)# 定义整个区域的相对介电常数（sympy）

# 定义输入（波导端口的数值解）
eigvals, eigvecs, yv = Laplacian_1D_eig(0, height, 1000, eps=eps_numpy, k=3)
yv = yv.reshape((-1, 1))
eigenmode = [1]
wave_number = 16.0  # wave_number = freq/c
waveguide_port_invar_numpy = {"x": np.zeros_like(yv), "y": yv}
waveguide_port_outvar_numpy = {"u": 10 * eigvecs[:, 0:1]}

#### Geo

In [7]:
rec = Rectangle((0, 0), (width, height))

#### PDE

In [8]:
hm = HelmholtzEquation(u="u", k=wave_number * eps_sympy, dim=2)  # 亥姆霍兹方程
gn = GradNormal(T="u", dim=2, time=False)  # 用于计算ABC

#### Model

In [9]:
# 定义简单的全连接网络
# 输入为空间坐标和
# 输出为E_z(x, y)
wave_net = instantiate_arch(
    input_keys=[Key("x"), Key("y")],
    output_keys=[Key("u")],
    frequencies=(
        "axis,diagonal",
        [i / 2.0 for i in range(int(wave_number * np.sqrt(eps1)) * 2 + 1)],
    ),
    frequencies_params=(
        "axis,diagonal",
        [i / 2.0 for i in range(int(wave_number * np.sqrt(eps1)) * 2 + 1)],
    ),
    cfg=cfg.arch.modified_fourier,
)

#### Node

In [10]:
nodes = (
    hm.make_nodes() + gn.make_nodes() + [wave_net.make_node(name="wave_network")]
)

#### Domain

In [11]:
waveguide_domain = Domain()

#### 入口条件（左边界）

In [12]:
Waveguide_port = PointwiseConstraint.from_numpy(
    nodes=nodes,
    invar=waveguide_port_invar_numpy,
    outvar=waveguide_port_outvar_numpy,
    batch_size=cfg.batch_size.Waveguide_port,
    lambda_weighting={"u": np.full_like(yv, 0.5)},
)
waveguide_domain.add_constraint(Waveguide_port, "Waveguide_port")

#### 右侧ABC边界

In [13]:
ABC = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={"normal_gradient_u": 0.0},
    batch_size=cfg.batch_size.ABC,
    lambda_weighting={"normal_gradient_u": 10.0},
    criteria=Eq(x, width),
)
waveguide_domain.add_constraint(ABC, "ABC")

#### 上下PEC边界

In [14]:
PEC = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={"u": 0.0},
    batch_size=cfg.batch_size.PEC,
    lambda_weighting={"u": 100.0},
    criteria=Or(Eq(y, 0), Eq(y, height)),
)
waveguide_domain.add_constraint(PEC, "PEC")

#### PDE约束

In [15]:
Interior = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={"helmholtz": 0.0},
    batch_size=cfg.batch_size.Interior,
    lambda_weighting={
        "helmholtz": 1.0 / wave_number**2,
    },
)
waveguide_domain.add_constraint(Interior, "Interior")

#### 验证器以及其他必要组件

In [16]:
slab_inference = VoxelInferencer(
    bounds=[[0, 2], [0, 2]],
    npoints=[256, 256],
    nodes=nodes,
    output_names=["u"],
    plotter=InferencerPlotter(),
)
waveguide_domain.add_inferencer(slab_inference, "Inf" + str(int(wave_number)))

### 求解器以及求解

In [17]:
# 定义求解器
slv = Solver(cfg, waveguide_domain)

手动加载日志系统

In [18]:
import logging
# logging.getLogger().addHandler(logging.StreamHandler())
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

启动求解

In [19]:
slv.solve()

2024-03-01 13:01:51,093 - WARNING - Installed PyTorch version 2.2.0a0+81ea7a4 is not TorchScript supported in Modulus. Version 2.1.0a0+4136153 is officially supported.
2024-03-01 13:01:51,102 - INFO - attempting to restore from: /workspace/19_2D_DielectricSlabWaveguide/outputs
2024-03-01 13:01:51,103 - WARNING - optimizer checkpoint not found
2024-03-01 13:01:51,104 - WARNING - model wave_network.0.pth not found
2024-03-01 13:01:52,348 - INFO - [step:          0] record constraint batch time:  1.240e-01s
2024-03-01 13:01:53,866 - INFO - [step:          0] record inferencers time:  1.515e+00s
2024-03-01 13:01:53,951 - INFO - [step:          0] saved checkpoint to /workspace/19_2D_DielectricSlabWaveguide/outputs
2024-03-01 13:01:53,953 - INFO - [step:          0] loss:  1.637e+06
2024-03-01 13:01:56,540 - INFO - Attempting cuda graph building, this may take a bit...
2024-03-01 13:02:13,849 - INFO - [step:        100] loss:  1.152e+04, time/iteration:  1.990e+02 ms
2024-03-01 13:02:22,242

2024-03-01 13:10:04,991 - INFO - [step:       5900] loss:  2.988e+01, time/iteration:  8.601e+01 ms
2024-03-01 13:10:14,799 - INFO - [step:       6000] record constraint batch time:  1.310e-01s
2024-03-01 13:10:16,551 - INFO - [step:       6000] record inferencers time:  1.747e+00s
2024-03-01 13:10:16,630 - INFO - [step:       6000] saved checkpoint to /workspace/19_2D_DielectricSlabWaveguide/outputs
2024-03-01 13:10:16,631 - INFO - [step:       6000] loss:  2.679e+01, time/iteration:  1.164e+02 ms
2024-03-01 13:10:25,177 - INFO - [step:       6100] loss:  2.304e+01, time/iteration:  8.545e+01 ms
2024-03-01 13:10:32,828 - INFO - [step:       6200] loss:  2.243e+01, time/iteration:  7.648e+01 ms
2024-03-01 13:10:38,740 - INFO - [step:       6300] loss:  2.751e+01, time/iteration:  5.910e+01 ms
2024-03-01 13:10:43,743 - INFO - [step:       6400] loss:  2.545e+01, time/iteration:  4.999e+01 ms
2024-03-01 13:10:48,745 - INFO - [step:       6500] loss:  2.149e+01, time/iteration:  5.000e+01

2024-03-01 13:18:17,410 - INFO - [step:      12000] loss:  4.913e+00, time/iteration:  7.431e+01 ms
2024-03-01 13:18:22,815 - INFO - [step:      12100] loss:  4.534e+00, time/iteration:  5.403e+01 ms
2024-03-01 13:18:30,330 - INFO - [step:      12200] loss:  5.150e+00, time/iteration:  7.513e+01 ms
2024-03-01 13:18:38,719 - INFO - [step:      12300] loss:  4.643e+00, time/iteration:  8.386e+01 ms
2024-03-01 13:18:47,132 - INFO - [step:      12400] loss:  4.514e+00, time/iteration:  8.409e+01 ms
2024-03-01 13:18:55,861 - INFO - [step:      12500] loss:  4.102e+00, time/iteration:  8.727e+01 ms
2024-03-01 13:19:04,425 - INFO - [step:      12600] loss:  4.382e+00, time/iteration:  8.562e+01 ms
2024-03-01 13:19:12,777 - INFO - [step:      12700] loss:  4.455e+00, time/iteration:  8.350e+01 ms
2024-03-01 13:19:21,341 - INFO - [step:      12800] loss:  4.327e+00, time/iteration:  8.561e+01 ms
2024-03-01 13:19:29,969 - INFO - [step:      12900] loss:  4.119e+00, time/iteration:  8.627e+01 ms


2024-03-01 13:26:31,187 - INFO - [step:      18400] loss:  1.519e+00, time/iteration:  5.015e+01 ms
2024-03-01 13:26:36,132 - INFO - [step:      18500] loss:  1.822e+00, time/iteration:  4.944e+01 ms
2024-03-01 13:26:41,101 - INFO - [step:      18600] loss:  1.682e+00, time/iteration:  4.966e+01 ms
2024-03-01 13:26:46,145 - INFO - [step:      18700] loss:  1.571e+00, time/iteration:  5.043e+01 ms
2024-03-01 13:26:51,435 - INFO - [step:      18800] loss:  1.521e+00, time/iteration:  5.287e+01 ms
2024-03-01 13:26:56,534 - INFO - [step:      18900] loss:  1.696e+00, time/iteration:  5.098e+01 ms
2024-03-01 13:27:02,431 - INFO - [step:      19000] record constraint batch time:  1.169e-01s
2024-03-01 13:27:04,481 - INFO - [step:      19000] record inferencers time:  2.046e+00s
2024-03-01 13:27:04,549 - INFO - [step:      19000] saved checkpoint to /workspace/19_2D_DielectricSlabWaveguide/outputs
2024-03-01 13:27:04,551 - INFO - [step:      19000] loss:  1.504e+00, time/iteration:  8.015e+01

2024-03-01 13:32:06,471 - INFO - [step:      24800] loss:  5.035e-01, time/iteration:  4.936e+01 ms
2024-03-01 13:32:11,896 - INFO - [step:      24900] loss:  5.191e-01, time/iteration:  5.422e+01 ms
2024-03-01 13:32:17,364 - INFO - [step:      25000] record constraint batch time:  1.531e-01s
2024-03-01 13:32:19,013 - INFO - [step:      25000] record inferencers time:  1.646e+00s
2024-03-01 13:32:19,069 - INFO - [step:      25000] saved checkpoint to /workspace/19_2D_DielectricSlabWaveguide/outputs
2024-03-01 13:32:19,071 - INFO - [step:      25000] loss:  4.798e-01, time/iteration:  7.172e+01 ms
2024-03-01 13:32:23,972 - INFO - [step:      25100] loss:  4.422e-01, time/iteration:  4.900e+01 ms
2024-03-01 13:32:28,848 - INFO - [step:      25200] loss:  4.517e-01, time/iteration:  4.874e+01 ms
2024-03-01 13:32:33,859 - INFO - [step:      25300] loss:  5.128e-01, time/iteration:  5.007e+01 ms
2024-03-01 13:32:38,687 - INFO - [step:      25400] loss:  4.503e-01, time/iteration:  4.826e+01

...

2024-03-01 19:06:34,914 - INFO - [step:     398900] loss:  4.328e-04, time/iteration:  4.539e+01 ms
2024-03-01 19:06:39,558 - INFO - [step:     399000] record constraint batch time:  1.002e-01s
2024-03-01 19:06:40,985 - INFO - [step:     399000] record inferencers time:  1.425e+00s
2024-03-01 19:06:41,055 - INFO - [step:     399000] saved checkpoint to /workspace/19_2D_DielectricSlabWaveguide/outputs
2024-03-01 19:06:41,057 - INFO - [step:     399000] loss:  3.339e-04, time/iteration:  6.140e+01 ms
2024-03-01 19:06:45,593 - INFO - [step:     399100] loss:  8.940e-03, time/iteration:  4.536e+01 ms
2024-03-01 19:06:50,158 - INFO - [step:     399200] loss:  3.611e-04, time/iteration:  4.563e+01 ms
2024-03-01 19:06:54,756 - INFO - [step:     399300] loss:  5.290e-03, time/iteration:  4.596e+01 ms
2024-03-01 19:06:59,365 - INFO - [step:     399400] loss:  3.462e-03, time/iteration:  4.606e+01 ms
2024-03-01 19:07:03,968 - INFO - [step:     399500] loss:  3.092e-03, time/iteration:  4.601e+01

2024-03-01 19:11:28,635 - INFO - [step:     405000] loss:  3.255e-03, time/iteration:  6.120e+01 ms
2024-03-01 19:11:33,256 - INFO - [step:     405100] loss:  5.053e-04, time/iteration:  4.620e+01 ms
2024-03-01 19:11:37,890 - INFO - [step:     405200] loss:  3.471e-04, time/iteration:  4.632e+01 ms
2024-03-01 19:11:42,635 - INFO - [step:     405300] loss:  4.095e-03, time/iteration:  4.743e+01 ms
2024-03-01 19:11:47,259 - INFO - [step:     405400] loss:  3.936e-04, time/iteration:  4.622e+01 ms
2024-03-01 19:11:51,839 - INFO - [step:     405500] loss:  3.110e-04, time/iteration:  4.580e+01 ms
2024-03-01 19:11:56,771 - INFO - [step:     405600] loss:  2.084e-03, time/iteration:  4.930e+01 ms
2024-03-01 19:12:01,331 - INFO - [step:     405700] loss:  4.722e-03, time/iteration:  4.559e+01 ms
2024-03-01 19:12:05,891 - INFO - [step:     405800] loss:  3.328e-03, time/iteration:  4.559e+01 ms
2024-03-01 19:12:10,460 - INFO - [step:     405900] loss:  8.916e-04, time/iteration:  4.567e+01 ms


2024-03-01 19:16:34,097 - INFO - [step:     411400] loss:  1.548e-03, time/iteration:  4.546e+01 ms
2024-03-01 19:16:38,637 - INFO - [step:     411500] loss:  4.934e-04, time/iteration:  4.539e+01 ms
2024-03-01 19:16:43,588 - INFO - [step:     411600] loss:  1.133e-02, time/iteration:  4.949e+01 ms
2024-03-01 19:16:48,202 - INFO - [step:     411700] loss:  4.199e-03, time/iteration:  4.612e+01 ms
2024-03-01 19:16:52,872 - INFO - [step:     411800] loss:  2.250e-03, time/iteration:  4.669e+01 ms
2024-03-01 19:16:57,448 - INFO - [step:     411900] loss:  4.541e-04, time/iteration:  4.574e+01 ms
2024-03-01 19:17:02,106 - INFO - [step:     412000] record constraint batch time:  1.047e-01s
2024-03-01 19:17:03,441 - INFO - [step:     412000] record inferencers time:  1.332e+00s
2024-03-01 19:17:03,509 - INFO - [step:     412000] saved checkpoint to /workspace/19_2D_DielectricSlabWaveguide/outputs
2024-03-01 19:17:03,511 - INFO - [step:     412000] loss:  1.329e-02, time/iteration:  6.061e+01

2024-03-01 19:21:41,927 - INFO - [step:     417800] loss:  2.835e-04, time/iteration:  4.622e+01 ms
2024-03-01 19:21:46,542 - INFO - [step:     417900] loss:  1.466e-03, time/iteration:  4.614e+01 ms
2024-03-01 19:21:51,225 - INFO - [step:     418000] record constraint batch time:  6.533e-02s
2024-03-01 19:21:52,543 - INFO - [step:     418000] record inferencers time:  1.316e+00s
2024-03-01 19:21:52,615 - INFO - [step:     418000] saved checkpoint to /workspace/19_2D_DielectricSlabWaveguide/outputs
2024-03-01 19:21:52,617 - INFO - [step:     418000] loss:  2.459e-04, time/iteration:  6.073e+01 ms
2024-03-01 19:21:57,240 - INFO - [step:     418100] loss:  1.356e-03, time/iteration:  4.622e+01 ms
2024-03-01 19:22:01,880 - INFO - [step:     418200] loss:  4.312e-04, time/iteration:  4.638e+01 ms
2024-03-01 19:22:06,452 - INFO - [step:     418300] loss:  5.049e-03, time/iteration:  4.570e+01 ms
2024-03-01 19:22:11,025 - INFO - [step:     418400] loss:  4.703e-04, time/iteration:  4.572e+01

2024-03-01 19:26:40,162 - INFO - [step:     424000] saved checkpoint to /workspace/19_2D_DielectricSlabWaveguide/outputs
2024-03-01 19:26:40,164 - INFO - [step:     424000] loss:  9.158e-04, time/iteration:  6.148e+01 ms
2024-03-01 19:26:44,742 - INFO - [step:     424100] loss:  1.212e-02, time/iteration:  4.577e+01 ms
2024-03-01 19:26:49,320 - INFO - [step:     424200] loss:  3.013e-04, time/iteration:  4.576e+01 ms
2024-03-01 19:26:53,893 - INFO - [step:     424300] loss:  9.410e-03, time/iteration:  4.572e+01 ms
2024-03-01 19:26:58,464 - INFO - [step:     424400] loss:  1.774e-03, time/iteration:  4.569e+01 ms
2024-03-01 19:27:03,026 - INFO - [step:     424500] loss:  2.048e-03, time/iteration:  4.562e+01 ms
2024-03-01 19:27:07,937 - INFO - [step:     424600] loss:  4.122e-04, time/iteration:  4.910e+01 ms
2024-03-01 19:27:12,510 - INFO - [step:     424700] loss:  3.743e-04, time/iteration:  4.572e+01 ms
2024-03-01 19:27:17,147 - INFO - [step:     424800] loss:  1.318e-02, time/iter

### 后处理以及可视化

对于jupyter，比较方便的方法是使用matplotlib

此外，还可以使用tensorboard以及Paraview

![u](./outputs/inferencers/Inf16_u.png)